 Detalhamento Técnico dos Módulos
A seguir, descrevo a responsabilidade técnica de cada componente, conectando com os conceitos teóricos de PLN fornecidos na sua base de conhecimento.

1. Módulo collectors (A Etapa de Extração)
Este é o ponto de entrada dos dados. Conforme discutido na teoria, a extração pode ser considerada a primeira etapa do pré-processamento.

base_scraper.py: Implementa a ética de scraping. Deve conter o mecanismo de time.sleep (Rate Limiting) e a rotação de User-Agent para emular navegadores reais.

html_scraper.py:

Usa requests para baixar o HTML bruto.

Usa BeautifulSoup para navegar na árvore DOM e identificar tags de conteúdo.


Ponto Crítico: Sites dinâmicos (que carregam dados via JavaScript, comuns em dashboards do INPE) exigirão o uso de Selenium ou Playwright.

pdf_extractor.py: Focado em documentos técnicos (Legislação/Relatórios). Ferramentas como pdfplumber são essenciais aqui para extrair texto mantendo a estrutura tabular, que é comum em relatórios de focos de calor.

2. Módulo processors (O Coração da Sanitização)
Aqui transformamos dados brutos em um Corpus de alta qualidade. Esta etapa é crítica para remover ruídos que confundem o modelo.

cleaner.py (Remoção de Marcação e Ruído):

Remove tags HTML, scripts e estilos CSS, restando apenas o texto visível.
+1

Utiliza Expressões Regulares (Regex) para limpar "sujeira" visual ou padrões de formatação irrelevantes, como quebras de linha excessivas ou caracteres de controle.

Boilerplate Removal: Remove cabeçalhos, rodapés, menus de navegação e propagandas, focando no conteúdo "core" do artigo jornalístico ou técnico.

normalizer.py (Padronização):

Normalização de Caixa: Decide estrategicamente quando usar lower case. Para modelos modernos (como BERT/GPT), manter a caixa (Cased) é importante para Reconhecimento de Entidades Nomeadas (NER), como distinguir "Mata" (bioma) de "mata" (verbo).


Caracteres Especiais: Trata codificações Unicode (UTF-8) para evitar erros de leitura (mojibake).

tokenizer.py (Segmentação):

Divide o texto contínuo em sentenças e palavras. Esta é a base para que o modelo entenda onde começa e termina uma ideia.

Utilizaremos bibliotecas robustas como NLTK ou Spacy para garantir que abreviações (ex: "Sra.", "art. 5º") não sejam confundidas com fim de frase.

3. Módulo storage (Persistência)
O armazenamento deve ser estruturado para permitir consultas rápidas e rastreabilidade.

Integração com SQL: O arquivo banco.sql fornecido sugere uma estrutura relacional robusta.

Podemos mapear os metadados extraídos dos PDFs/Sites (Data, Localização, Fonte) para as tabelas evento_fogo.focos_bdq_c2 ou evento_fogo.evento_fogo.

Exemplo: Ao extrair uma notícia sobre incêndio no Pantanal, o scraper deve tentar popular os campos data_hora_gmt, municipio, e bioma da tabela de focos.

 Fluxo de Trabalho (Pipeline) Sugerido
Baseado no diagrama de pipeline de PLN, seu fluxo de dados será:

Ingestão: O main.py lê sources.json, ativa o html_scraper.py.

Raw Storage: O HTML bruto é salvo em data/01_raw. Motivo: Se o parser falhar, não precisamos baixar o site de novo.


Parsing/Cleaning: O cleaner.py carrega o HTML bruto, remove tags, e extrai o texto limpo.


Segmentation: O tokenizer.py quebra o texto em sentenças.

Metadata Extraction: Regex e NER identificam datas e locais.

Loading: Os dados estruturados vão para o PostgreSQL e o texto limpo para data/03_processed para o treino do modelo.

etapa do pipeline descrita na teoria de PLN:

JSON (.json): Ideal para a Etapa de Extração (Raw Data). Ele preserva a estrutura hierárquica (metadados + conteúdo) e lida bem com caracteres especiais e aninhamento, algo essencial para manter o "texto bruto" intacto.

TXT (.txt): Ideal para o Corpus Processado. A maioria dos modelos de linguagem (LLMs) e bibliotecas como NLTK/Spacy consomem texto plano para treinamento e tokenização.

CSV (.csv): Ideal apenas para Dados Tabulares extraídos de relatórios (ex: tabelas de focos de calor por município), pois facilita a leitura por bibliotecas como Pandas.